# Taller de Reglas de Asociación

Los algoritmos de reglas de asociación se utilizan para encontrar hechos o elementos que ocureen en comñun dentro de un determinado conjunto de datos. Se definen como reglas de implicación del tipo <b>"si X entonces Y" (X --> Y)</b> donde <i>X</i> es el antecedente o LHS y <i>Y</i> es el consecuente o RHS.
Ejemplo: La regla <b>{A, B} --> C</b> indica que si A y B ocurren en un conjunto de items, entonces C también ocurre en ese conjunto. El lado izquiero de la regla recibe el nombre de antecedente o Left Hand Side (LHS) y el lado derecho de la regla recibe el nombre de consecuente o Right Hand Side (RHS).

## Conceptos importantes
<ul>
<li><b>Ítem:</b> cada uno de los elementos que componen una transacción.</li>

<li><b>Itemset:</b> conjunto de ítems. Un k-itemset es un itemset con k items.</li>

<li><b>Transacción:</b> conjunto de ítems vinculados a un evento concreto y con un identificador único.</li>
</ul>

### Imports

In [ ]:
import pandas as pd
from apyori import apriori

### Carga de archivos

In [ ]:
recetados_df = pd.read_csv("medicamentos_recetados.csv")
dimension_df = pd.read_csv("dim_medicamento.csv")

In [ ]:
print(recetados_df.info())
print(dimension_df.info())

### Transformación de datos

In [ ]:
merged_df = pd.merge(
    recetados_df, 
    dimension_df[['key_medicamento', 'Nombre Generico']], 
    on='key_medicamento', 
    how='inner'
)

In [ ]:
merged_df.head()

In [ ]:
medications_gropued_by_formula = merged_df.groupby('codigo_formula')['Nombre Generico'].apply(list)
transactions = medications_gropued_by_formula.tolist()
medications_gropued_by_formula.to_csv('assets/result_files/medications_grouped_by_formula.csv')

In [ ]:
print("medicamentos_agrupados_por_formula")
print(medications_gropued_by_formula)
print("transacciones_lista_por_formula")
print(transactions)

In [ ]:
print(transactions[122])

### Algoritmo Apriori

Es un algoritmo propuesto por Adrwal y Srikant en 1994.
#### Reglas de asociación utlizadas
##### Soporte
Para un item o itemset X, el soporte es el número de transacciones que contienen X, divido entre el total de transacciones.

Se tiene la siguiente ecuación:

<p align="center">
  <img src="assets/images/support-ec.png" alt="support-ec" width="500"/>
</p>

##### Confianza
Se tiene que X --> Y, entonces la confianza se define siguiendo la ecuación:

<p align="center">
  <img src="assets/images/confidence-ec.png" alt="confidence-ec" width="500"/>
</p>

donde union(X,Y) es el itemset que contiene todos los items de X y Y. La confianza se interpreta como la probabilidad de que una transaccion que contiene los items de X, tambien contiene los items de Y.

##### Lift
Mide la frecuencia son la que se encuentra el consecuente Y de cuando el antecendete A esta presenta dentro de una asociación X --> Y, comparado a lo que deberia ocurrir si fueran estadisticamente independientes. En otras palabras, compara la frecuencia observada del patrón con lo que se esperaría solo por azar. Se tiene la siguiente ecuación:

<p align="center">
  <img src="assets/images/lift-ec.png" alt="lift-ec" width="300"/>
</p>

##### Ejecución manual de las reglas de asociación

Cambiar lhs_med y rhs_med por las dos medicaciones que se quiere probar manualmente.

In [ ]:
total_transactions = len(transactions)
lhs_med = 'AMIKACINA'
rhs_med = 'VANCOMICINA'

count_both = sum([lhs_med in t and rhs_med in t for t in transactions])
print("Transacciones ambas:", count_both)

count_lhs = sum([lhs_med in t for t in transactions])
print("Transacciones izquierda:", count_lhs)

count_rhs = sum([rhs_med in t for t in transactions])
print("Transacciones derecha:", count_rhs)

support_lhs = count_lhs / total_transactions
print("Soporte izquierda:", support_lhs)

support_rhs = count_rhs / total_transactions
print("Soporte derecha:", support_rhs)

support_manual = count_both / total_transactions
print("Soporte manual:", support_manual)

confidence_manual = count_both / count_lhs if count_lhs > 0 else 0
print("Confidencia manual:", confidence_manual)

lift_manual = confidence_manual / support_rhs if count_rhs > 0 else 0
print("Lift manual:", lift_manual)


In [ ]:
rules = apriori(
    transactions,
    min_support=0.0005,
    min_confidence=0.3,
    min_lift=0.1,
    min_length=2,
)

results = list(rules)

Aqui se retorna un DataFrame con las reglas de asociación de medicaciones, aún no se pueden visualizar correctamente.

In [ ]:
rules_df = pd.DataFrame(results)
rules_df.head(30)

En esta sección se inspecciona el dataFrame results para mostrar los medicamentos y los datos obtenidos para cada asociación.

In [ ]:
def inspect(output):
    lhs = [tuple(result[2][0][0])[0] for result in output if len(result[2]) > 0]
    rhs = [tuple(result[2][0][1])[0] for result in output if len(result[2]) > 0]
    support = [result[1] for result in output if len(result[2]) > 0]
    confidence = [result[2][0][2] for result in output if len(result[2]) > 0]
    lift = [result[2][0][3] for result in output if len(result[2]) > 0]
    return list(zip(lhs, rhs, support, confidence, lift))


transactions_with_associations_df = pd.DataFrame(inspect(results), columns=['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])
transactions_with_associations_df.head(10)

Guardamos en un csv y verificamos la cantidad de filas y columnas.

In [ ]:
transactions_with_associations_df.to_csv('assets/result_files/transactions_with_associations.csv', index=False)
transactions_with_associations_df.shape

In [ ]:
transactions_with_associations_df.nlargest(n=119, columns='Confidence')

Es posible que algunas asociaciones aparezcan duplicadas, por lo que limpiamos la tabla filtrando los duplicados.

In [ ]:
transactions_with_associations_df_cleaned = transactions_with_associations_df[transactions_with_associations_df['Right Hand Side'] != 'nan']
transactions_with_associations_df_cleaned = transactions_with_associations_df_cleaned.drop_duplicates(
    subset=['Left Hand Side', 'Right Hand Side'], keep='first')
transactions_with_associations_df_cleaned.nlargest(n=111, columns= 'Confidence')

Guardamos en otro csv y verificamos las filas y columnas que quedan.

In [ ]:
transactions_with_associations_df_cleaned.to_csv('assets/result_files/transactions_with_associations_cleaned.csv', index=False)
transactions_with_associations_df_cleaned.shape

Ahora ordenamos por confianza y si hay igualdad, por soporte.

In [ ]:
#transactions_with_associations_df_cleaned.nlargest(n=40, columns= 'Confidence')
sorted_transactions_with_associations_df_cleaned = transactions_with_associations_df_cleaned.sort_values(
    by=['Confidence', 'Support'], ascending=[False, False])

sorted_transactions_with_associations_df_cleaned.head(30)

## ¿Qué significan los resultados?
En este dataFrame ordenamos las asociaciones de medicamentos por confianza, por lo que se puede afirmar con una confianza muy alta que, si se compra el medicamento de la izquierda, se compra el medicamento de la derecha.

Como criterio de desempate, se considera el soporte de la asociación, es decir, el número de veces que se llevan estos medicamentos sobre el total de transacciones.